In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import networkx as nx

from entsoe import EntsoePandasClient
from entsoe.mappings import Area, NEIGHBOURS, lookup_area
from entsoe.parsers import parse_generation
from typing import Union, Optional, Dict, List, Literal

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [2]:
zones = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [3]:
df_gen = pd.read_excel('../cost_curves/data/generation.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)
df_powerplants = pd.read_excel('./plants.xlsx', index_col=0)
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_grid = df_grid[df_grid.susceptance != 0]
Grid = nx.from_pandas_edgelist(df_grid, 'substation_1', 'substation_2', edge_attr=["susceptance"])

substation_ids = []
substation_locs = []
for index, row in df_substations.iterrows():
    substation_ids.append(index)
    substation_locs.append((row.lon, row.lat))
    
substation_tree = spatial.KDTree(substation_locs)

for index, row in df_powerplants.iterrows():
    result = substation_tree.query([(row.lon,row.lat)])
    df_powerplants.loc[index, 'node'] = substation_ids[result[1][0]]
    
df_powerplants.node = df_powerplants.node.astype(int)
display(df_powerplants)

,name,eic,type,zone,installed_capacity,lat,lon,node
0,GDK-Mellach,14W-WGM-KW-----J,natural_gas,AT,838.0,46.911700,15.488300,1091
1,Ottensheim-Wilhering,14W-BOW-KW-----9,hydro_run_of_river,AT,179.0,48.316500,14.151057,1280
2,FHKW Linz Mitte,14WLSG-MITTE---U,natural_gas,AT,216.8,48.299219,14.322935,690
3,Kraftwerk Theiß,14W-KW-THE-EVN-7,natural_gas,AT,485.0,48.394861,15.710147,1064
4,FHKW Linz Süd,14WLSG-1SUED---5,natural_gas,AT,158.0,48.267893,14.346898,690
...,...,...,...,...,...,...,...,...
636,Petrokemija Kutina,NaN,natural_gas,HR,35.0,45.471290,16.792320,1171
637,Dubrovnik,NaN,hydro_reservoir,HR,216.0,42.603001,18.235001,187
638,Fuzine,NaN,hydro_pumped_storage,HR,5.7,45.304985,14.714837,412
639,He Rijeka,NaN,hydro_run_of_river,HR,36.8,45.334999,14.452000,417


In [4]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
#end = pd.Timestamp('20220902', tz='Europe/Amsterdam')
end = pd.Timestamp('20230331', tz='Europe/Amsterdam')

df_actual_generation = {}

for (index, country_code) in enumerate([x for x in zones if x not in ['ALBE', 'ALDE', 'DE_LU', 'HR']]):
    print(country_code)
    df_actual_generation[country_code] = client.query_generation_per_plant(country_code, start=start, end=end)
    

AT
BE


KeyboardInterrupt: 

In [60]:
display(df_actual_generation['AT'].iloc[0])

                                                         newlevel         
Abwinden-Asten          Hydro Run-of-river and poundage  Actual Aggregated     93.0
Altenwörth              Hydro Run-of-river and poundage  Actual Aggregated    177.0
Aschach                 Hydro Run-of-river and poundage  Actual Aggregated    131.0
Block 07 Linz           Fossil Gas                       Actual Aggregated    124.0
DO3                     Fossil Gas                       Actual Aggregated      0.0
FHKW Mellach            Fossil Gas                       Actual Aggregated      0.0
Freudenau               Hydro Run-of-river and poundage  Actual Aggregated     98.0
GDK-Mellach Linie 10    Fossil Gas                       Actual Aggregated      0.0
GDK-Mellach Linie 20    Fossil Gas                       Actual Aggregated      0.0
Gerlos                  Hydro Water Reservoir            Actual Aggregated     24.0
Greifenstein            Hydro Run-of-river and poundage  Actual Aggregated    156.0
H

In [5]:
gsk5_matrix = np.zeros((len(df_gen), len(df_powerplants), len(zones)))

for p, (p_index, plant) in enumerate(df_powerplants.iterrows()):
    if plant['zone'] == 'AT':
        zone = plant['zone']
        if plant['name'] in df_actual_generation[zone].columns:
            print(p)
            for t, (t_index, gen) in enumerate(df_gen.iterrows()):
                gen_level = list(df_actual_generation[zone][plant['name']][df_actual_generation[zone][plant['name']].columns[0]])[t]
                total_zonal_generation = df_actual_generation[zone].iloc[t].sum()

                gsk5_matrix[t][p][zones.index(zone)] = gen_level/total_zonal_generation

1
6
7
9
10
11
13
15
16
17
19
20
23
24
25
26
29
32
33
34
35


In [6]:
with open('gsk5_matrix_p.npy', 'wb') as f:
    np.save(f, gsk5_matrix)

In [7]:
gsk5_matrix[0]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05251641, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
len(df_gen)

5064